**Treinamento de classificador** para realização de análise de sentimentos (modelo baseado em atividade da disciplina de PLN - PUCPR).
Iremos desenvolver o classificador, utilizando uma abordagem supervisionada, ou seja, precisaremos de dados rotulados com suas respectivas emoções.

#### Fluxo de execução
Vamos seguir o seguinte fluxo de processamento dos dados:


1.   Abrir o corpus
2.   Remover as stop-words
3.   Aplicar stemmer
4.   Gerar o Bag of Words
5.   Treinar o modelo SVM
6.   Predizer/Avaliar o modelo



**1) Primeira parte** - Nesse início de execução do algoritmo, foi realizado a **Redução da granularidade dos sentimentos**

In [ ]:
# Abre corpus para leitura
f = open("analise-sentimentos-2000-noticias.txt", "r", encoding="utf-8-sig")

#Variável que recebe todas as linhas do doc aberto anteriormente
linhas = f.readlines()

#Variável criada para receber as linhas sem o rótulo 'surpresa'
#Foi realizada uma iteração linha por linha para verificação
nova_lista = [item for item in linhas if not item.startswith("surpresa")]

#Variáveis que receberão a separação de rótulos e textos
corpus_textos = []
corpus_rotulos = []

# Percorre as linhas
for linha in nova_lista:

  # Separa texto e rótulo/categoria/emoção
  item = linha.split(";;")

  #adiciona às variáveis especificadas os rótulos e textos de cada linha
  corpus_rotulos.append(item[0])
  corpus_textos.append(item[1])

In [ ]:
#Nova lista criada sem as linhas do rótulo 'surpresa'
print(nova_lista)

In [ ]:
# 5 primeiros textos
corpus_textos[0:5]

In [ ]:
#Novo tamanho de texto sem os textos que tinham como rótulo 'surpresa'
len(corpus_textos)

In [ ]:
#Variável que contém os novos rótulos (positivo, negativo e neutro)
novos_rotulos = {"alegria":"positivo", "raiva":"negativo", "medo":"negativo", "desgosto":"negativo", "tristeza":"negativo", "neutro":"neutro"}

# Lista com os rótulos antigos
rotulos_antigos = corpus_rotulos

# Lista com os rótulos atualizados
rotulos_atualizados = [novos_rotulos[rotulo] for rotulo in rotulos_antigos]

#Exibe os novos rótulos
print(rotulos_atualizados)

In [ ]:
rotulos_atualizados[0:15]

In [ ]:
#Tamanho atualizado dos rótulos (mesmo tamanho dos textos)
len(rotulos_atualizados)

In [ ]:
corpus_rotulos[0:15]

In [ ]:
#Corpus com rótulos antigos usado para mostrar a atualização,
#ou seja, a retirada dos textos com rótulos de 'surpresa'
len(corpus_rotulos)

In [ ]:
from sklearn.model_selection import train_test_split

# O próprio sklearn tem um método para dividir a base de dados em treinamento e teste
# Neste caso estamos deixando 90% para treinamento e 10% para testes
corpus_treinamento, corpus_teste, rotulos_treinamento, rotulos_teste = train_test_split(corpus_textos, rotulos_atualizados, test_size=0.10, random_state=42)

In [ ]:
#Tamanho do corpus de treinamento
len(corpus_treinamento)

In [ ]:
#Tamanho do corpus de teste
len(corpus_teste)

In [ ]:
#Tamanho dos rótulos de treinamento
len(rotulos_treinamento)

In [ ]:
#Tamanho dos rótulos de teste
len(rotulos_teste)

Vamos deixar preparada uma função para pré-processar os textos, utilizando uma lista de stop-words com novos itens, o stemming e normalização dos textos.

In [ ]:
#Importação das bibliotecas usadas para pré-processamento de texto
import nltk
from nltk import tokenize
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('punkt')

stopwords = nltk.corpus.stopwords.words('portuguese') #carrega stopwords da lingua portuguesa disponíveis no NLTK
stopwords += (',','.','(',')','"',"'",'´','`','!','$','%','&','...','-',':',';','?','``','\'\'') #acrescenta simbolos
stopwords += ('a','e','i','o','u','A','E','I','O','U') #acrescenta também vogais

stemmer = nltk.stem.RSLPStemmer()

#Função de pré-processamento de texto (lowercase, tokenização, stopwords e stemmização)
def my_preprocessor(text):

    # Normaliza para minúsculas
    text=text.lower()

    # Tokeniza
    words = tokenize.word_tokenize(text, language='portuguese')
    # Remove stop-words
    words_no_stopwords = [word for word in words if not word in stopwords]
    # Aplica stemming
    stemmed_words=[stemmer.stem(word=word) for word in words_no_stopwords]
    return ' '.join(stemmed_words)

2) **Segunda parte** - **Configuração dos parâmetros de extração de atributos e do classificador**. Aqui foram alterados os parâmetros, tanto do CountVectorizer quanto do classificador (SVC) e foram encotrados algumas diferenças na acurácia do treinamento e teste. Deixarei os exeplos escritos abaixo e comentados.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer

# Primeiro aplica o BoW, depois envia dados ao classificador SVM
# (SEM retirada de stop-words e stemming)
#Testar cada linha com diferenças entre parâmetros

#Aqui, optei por inserir o parâmetro 'max_df' no CountVectorizer e manter o SVC como estava
sent_clf = Pipeline([('vect', CountVectorizer(max_df=0.50)),('clf', SVC(kernel='linear', C=1))])

#Nessa linha, optei por alterar o parâmetro 'C' do SVC para 2, houve pouca mudança na acurária em relação a linha de cima
#sent_clf = Pipeline([('vect', CountVectorizer()),('clf', SVC(kernel='linear', C=2))])

#Nessa linha, optei por alterar tanto o 'vect (max_df), quanto o parâmetro 'C' do SVC, houve um aumento da acurária em relação a linha de cima
#sent_clf = Pipeline([('vect', CountVectorizer(max_df=0.50)),('clf', SVC(kernel='linear', C=2))])

# (COM retirada de stop-words e stemming)
#sent_clf = Pipeline([('vect', CountVectorizer(preprocessor = my_preprocessor)),('clf', SVC(kernel='linear', C=1))])

#Aqui optei por adicionar ao vectorizer o ngram_range e alterar o parâmetro 'C' do SVC.
#Houve uma diminuição considerável na acurácia se comparado com as outras linhas acima
#sent_clf = Pipeline([('vect', CountVectorizer(preprocessor = my_preprocessor, ngram_range=(2,2))),('clf', SVC(kernel='linear', C=2))])

#Nesta linha, optei por alterar somento o parâmetro 'ngram_range' e houve melhora na acurácia em relação a linha de cima
#sent_clf = Pipeline([('vect', CountVectorizer(preprocessor = my_preprocessor, ngram_range=(1,1))),('clf', SVC(kernel='linear', C=1))])

3) **Terceira parte** - **Novas etapas de extração de atributos ou de pré-processamento**. Nesta etapa optei por importar o TfidfTransformer para transformar a matriz gerada pelo Vectorizer em em uma representação TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

#Adição do TF-IDF ao pipeline para transformação da matriz
#Optei por deixar comentado a linha abaixo para testar outro classificador
#sent_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', SVC(kernel='linear', C=1))])

4) **Quarta parte - Utilização de outro classificador de texto**. Aqui, optei por trocar SVC pelo MultinomialNB para classificar o texto. Houve diferenças na acurácia dos treinamento e testes realizados em comparação aos métodos testados acima.

In [ ]:
#importação do classificador de Naive Bayes
from sklearn.naive_bayes import MultinomialNB

#Substituição do classificador SVC pelo MultinomialNB
sent_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB())])

In [ ]:
# Inicia treinamento
sent_clf = sent_clf.fit(corpus_treinamento, rotulos_treinamento)

Já temos nosso modelo treinado! Agora vamos predizer a base de teste e avaliar os resultados.

In [ ]:
# Prediz base de teste
rotulos_preditos = sent_clf.predict(corpus_teste)

In [ ]:
from sklearn.metrics import classification_report

# Mostra relatório completo de avaliação
print(classification_report(rotulos_teste, rotulos_preditos))

In [ ]:
from sklearn.metrics import confusion_matrix

# Podemos imprimir a matriz de confusão para tentar entender melhor os resultados
mat = confusion_matrix(rotulos_teste, rotulos_preditos)

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

rotulos_nomes = ["negativo", "positivo", "neutro"]

fig, ax = plt.subplots(figsize=(5,5))
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False, xticklabels=rotulos_nomes, yticklabels=rotulos_nomes )
plt.xlabel('Categoria verdadeira')
plt.ylabel('Categoria predita');

**CONCLUSÃO** - Foi particularmente difícil realizar a primeira etapa, a da redução de granulidade, enquanto que as outras etapas foram mais tranquilas. Percebe-se a diferença de resultados de acurácia quando combinados diferentes parâmetros, classificadores e etapas de extração de atributos.